In [3]:
### IMPORTAMOS CÓDIGO PARA LAS PREGUNTAS DE TRIVIA
import requests
import html
import random
import sys
import time

from utiles.functions import get_session_token, reset_session_token, get_random_category_id, get_trivia_question, objeto_a_cat, new_question

### IMPORTAMOS CÓDIGO PARA LA DETECCIÓN DE IMAGEN
import cv2
import torch
import numpy as np

from utiles.functions import detectar_objeto, detectar_objeto_camara, detectar_objeto_imagen, ingresar_objeto_manual, predicted_object

#### IMPORTAMOS CÓDIGO DEL JUEGO ###

from IPython.display import Image, display
import random

from utiles.functions import linebreak, show_image, initial_settings, play_room, explore_room, examine_item, pet_friendly, start_game, get_next_room_of_door, final_test, end_game

In [5]:
# Definimos la relación de objetos en habitaciones para la búsqueda.

#DOORS:

door_library = {
    "name": "library door",
    "type": "door",
}

door_kitchen = {
    "name": "kitchen door",
    "type": "door",
}

door_living = {
    "name": "living room door",
    "type": "door",
}

door_bedroom = {
    "name": "bedroom door",
    "type": "door",
}

door_outside = {
    "name": "outside door",
    "type": "door",
}


#FURNITURE:


cup = {
    "name": "cup",
    "type": "furniture",
}

bottle = {
    "name": "bottle",
    "type": "furniture",
}

headphones = {
    "name": "headphones",
    "type": "furniture",
}

book = {
    "name": "book",
    "type": "furniture",
}

banana = {
    "name": "banana",
    "type": "furniture",
}

cat = {
    "name": "cat",
    "type": "furniture",
    'category': 'Animals',
}

dog = {
    "name": "dog",
    "type": "furniture",
    'category': 'Animals',
}

basketball = {
    "name": "basketball",
    "type": "furniture",
}

airplane = {
    "name": "airplane",
    "type": "furniture",
}

spacecraft = {
    "name": "Charlie's spacecraft",
    "type": "spacecraft",
}

#ROOMS:

hall = {
    "name": "hall",
    "type": "room",
    "image": 'url',
    "category": "General_Culture",
    "target": [cat,dog],
}

library = {
    "name": "library",
    "type": "room",
    "image": 'url',
    "category": 'Arts_and_Books',
    "target": [book],
    "clue": 'One of those blocks in the shelf would help me. Can you give one to me?',
}

kitchen = {
    "name": "kitchen",
    "type": "room",
    "image": 'url',
    "category": 'Science',
    "target": [banana],
    "clue": "My color is yellow, and I grow on trees, I'm a popular food with apes and monkeys"
}

living_room = {
    "name": "living room",
    "type": "room",
    "image": 'url',
    "category": 'History',
    "target": [airplane],
    "clue": 'What a beautiful model! Is that an Airbus A350? Show it to me, please!'
}

bedroom = {
  "name": "bedroom",
  "type": "room",
  "image": 'url',
  "category": 'Audiovisual',
  "target": [headphones],
  "clue": 'I love listening to music, do you have any item to help me listening to it without bothering anyone else?',
}

outside = {
  "name": "outside",
  "type": "room",
  "image": 'url',
  "category": 'Sports',
  "target": [basketball],
  "clue": 'It is important to do some exercise before a long trip. Wanna try some shots?'
}

space = {
  "name": "outer space",
  "type": "room",
  "image": 'url',
}

#Delimitar habitaciones:

all_rooms = [hall, library, kitchen, living_room, bedroom, outside]

all_doors = [door_library, door_kitchen, door_living, door_bedroom, door_outside]

# define which items/rooms are related

object_relations = {
    ### ROOMS
    "hall": [door_library, door_kitchen, door_living, door_bedroom, door_outside],
    "library": [book,  door_library],
    "kitchen": [banana, door_kitchen],
    "living room": [airplane, door_living],
    "bedroom": [headphones, door_bedroom],
    "outside": [basketball, door_outside, spacecraft],
    ### DOORS
    "library door": [hall, library],
    "kitchen door": [hall, kitchen],
    "living room door": [hall, living_room],
    "bedroom door": [hall, bedroom],
    "outside door": [hall, outside],
}

In [ ]:
game_state = initial_settings()

In [ ]:
start_game(game_state)

In [ ]:
### PRUEBECITA 

def detectar_objeto_camara2(objetivo):
    """
    Abre la webcam y detecta si el objeto especificado aparece en pantalla.
    Solo valida el objeto pasado como argumento.
    Si se detecta de forma estable en 5 frames consecutivos, confirma la detección.
    """
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    model.to('cpu')
    model.eval()
    # Iniciamos la cámara
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Webcam not available")
        return

    umbral_deteccion = 5
    timing = 0
    conteo_detectado = 0
    # Bucle para leer la cámara indefinidamente
    while True:
        timing +=1
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (320, 240))
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(img_rgb)
        detections = results.pandas().xyxy[0]

        cv2.imshow("Let's help Charlie!!", frame)
        encontrado = any(detections.name == objetivo)
        if encontrado:
            conteo_detectado += 1
        else:
            conteo_detectado = 0

        if conteo_detectado >= umbral_deteccion:
            print(f"{objetivo} correctly detected!")
            cap.release()
            cv2.destroyAllWindows()
            return objetivo
        elif timing >=100:
            print(f"{objetivo} not detected.")
            objetivo = 'Fallido'
            cap.release()
            cv2.destroyAllWindows()
            return objetivo 
        
        # Rompe el bucle si se presiona 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            objetivo = 'Fallido'
            cap.release()
            cv2.destroyAllWindows()
            return objetivo 

    return None

detectar_objeto_camara2('bottle')



Using cache found in C:\Users\Usuario/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-8 Python-3.12.9 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


'Fallido'